In [1]:
import os
import datetime as dt
from pyclarify import ClarifyClient, query
import orchest

credentials = os.environ["CLARIFY_CREDENTIALS"]
client = ClarifyClient(credentials)

In [2]:
ts_last_hours = orchest.get_step_param("ts_last_hours")

ts_start = (
    (dt.datetime.today() - dt.timedelta(hours=ts_last_hours))
    if ts_last_hours
    else dt.datetime(2021, 8, 10, 0)
)
item_id = orchest.get_step_param("item_id")

In [3]:
query_filter = query.Filter(fields={"id": query.In(value=[item_id])})
response = client.select_items(
    filter=query_filter,
    not_before=ts_start,  # minimum date
    before=dt.datetime.today(),  # maximum date
    include_metadata=False
)

In [4]:
signal = (
    response.result.data.to_pandas()
    [item_id]  # convert to Series
    .drop_duplicates()  # sanity check
    .asfreq("H")  # fill gaps
    .rename("traffic")  # use explicit name
)

In [5]:
signal.head()

2021-08-10 12:30:00+00:00    6565.0
2021-08-10 13:30:00+00:00    7361.0
2021-08-10 14:30:00+00:00    6162.0
2021-08-10 15:30:00+00:00    6242.0
2021-08-10 16:30:00+00:00    4990.0
Freq: H, Name: traffic, dtype: float64

In [6]:
orchest.output(signal, name="timeseries")